# 获取奖学金信息

### 环境准备

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

### 连接数据库

In [3]:
import sqlite3
conn = sqlite3.connect("../db.sqlite3")
cur = conn.cursor()
table_name = cur.execute("select name from sqlite_master where type='table' order by name").fetchall()
table_name

[('Training',),
 ('UserTemp',),
 ('account_confirmstring',),
 ('account_profile_awardhistory',),
 ('account_profile_awardhistory_User',),
 ('account_profile_competitionrecord',),
 ('account_profile_honortitle',),
 ('account_profile_paperrecord',),
 ('account_profile_patentrecord',),
 ('account_profile_publishrecord',),
 ('account_profile_researchproject',),
 ('account_user',),
 ('account_user_groups',),
 ('account_user_user_permissions',),
 ('auth_group',),
 ('auth_group_permissions',),
 ('auth_permission',),
 ('award_info_awardclassification',),
 ('award_info_awardinfo',),
 ('award_info_awardinfo_Major',),
 ('award_info_awardinfopost',),
 ('award_info_category',),
 ('award_info_post',),
 ('award_info_post_tags',),
 ('award_info_post_users_focus',),
 ('award_info_tag',),
 ('captcha_captchastore',),
 ('django_admin_log',),
 ('django_content_type',),
 ('django_migrations',),
 ('django_session',),
 ('grade_info_deptinfo',),
 ('grade_info_gradeinfo',),
 ('grade_info_lessoninfo',),
 ('grade

In [4]:
cur.execute("PRAGMA table_info(account_profile_awardHistory)")
cur.fetchall()

[(0, 'id', 'integer', 1, None, 1),
 (1, 'awardName', 'varchar(25)', 1, None, 0),
 (2, 'AwardID_id', 'integer', 1, None, 0),
 (3, 'AwardYear_id', 'integer', 1, None, 0)]

In [6]:
cur.execute("PRAGMA table_info(account_profile_awardHistory_User)")
cur.fetchall()

[(0, 'id', 'integer', 1, None, 1),
 (1, 'awardhistory_id', 'integer', 1, None, 0),
 (2, 'user_id', 'integer', 1, None, 0)]

### AwardHistory 的CSV

In [7]:
csv_head1 = ["id", "awardName"]
record = conn.execute("select {} from account_profile_awardHistory".format(",".join(csv_head1)))

In [14]:
import csv

def save_csv(record, csv_head, csv_name):
    with open(csv_name, "w", newline="") as f:
        ff = csv.writer(f)
        ff.writerow(csv_head)
        ff.writerows(record)


def read_record(csv_name, csv_head):
    df_record = pd.read_csv(csv_name, header=0, encoding="utf-8")
    mapping = list(map(lambda x:False if x < 1 else True, [i for i in range(0, len(csv_head))]))
#     df_one_hot = pd.factorize(df_record.loc[:, mapping].copy())
    df_fact = df_record.loc[:, mapping].copy()
    df_one_hot = pd.get_dummies(df_record.loc[:, mapping].copy())
    return df_record, df_one_hot, df_fact

In [12]:
save_csv(record, csv_head1, "award_awardName.csv")

In [15]:
df_record, df_record_one_hot, df_record_fact = read_record("award_awardName.csv", csv_head1)
df_record_fact.columns

Index(['awardName'], dtype='object')

In [16]:
for i in df_record_fact.columns:
    df_record_fact[i] = pd.Series(pd.factorize(df_record_fact[i])[0])

In [17]:
df_record_one_hot['id'] = df_record.loc[:, ["id"]]
df_record_one_hot = df_record_one_hot.sort_values("id").reset_index(drop=True)
df_record_one_hot.to_csv("awardname_one_hot.csv")

In [18]:
df_record_fact["id"] = df_record.loc[:, ['id']]
df_record_fact = df_record_fact.sort_values("id").reset_index(drop=True)
df_record_fact.to_csv("awardname_fact.csv")

### AwardUser 的CSV

In [29]:
csv_head2 = ["id", "awardhistory_id", "user_id"]
record = conn.execute("select {} from account_profile_awardHistory_User".format(",".join(csv_head2)))

In [20]:
save_csv(record, csv_head2, "award_user.csv")

In [30]:
user_award = record.fetchall()

In [26]:
df_record_fact

,awardName,id
0,0,1
1,1,2
2,2,3
3,3,4
4,2,5
...,...,...
179,3,180
180,3,181
181,3,182
182,3,183


In [35]:
df_user = pd.DataFrame(user_award, columns=["id", "award_id", "user_id"])

In [38]:
df_user = df_user.drop(["id"], axis=1)

In [39]:
df_user

,award_id,user_id
0,1,3900
1,2,3102
2,3,3163
3,4,4323
4,5,3037
...,...,...
179,180,3536
180,181,3516
181,182,3940
182,183,4228


In [40]:
df_record_fact.rename({"id": "award_id"}, axis=1, inplace=True)

In [41]:
df_record_fact

,awardName,award_id
0,0,1
1,1,2
2,2,3
3,3,4
4,2,5
...,...,...
179,3,180
180,3,181
181,3,182
182,3,183


In [44]:
df_user = df_user.merge(df_record_fact, how='inner', on='award_id')

In [45]:
df_user

,award_id,user_id,awardName
0,1,3900,0
1,2,3102,1
2,3,3163,2
3,4,4323,3
4,5,3037,2
...,...,...,...
179,180,3536,3
180,181,3516,3
181,182,3940,3
182,183,4228,3


In [48]:
df_user.groupby('awardName').count()

,award_id,user_id
awardName,,
0,4,4
1,27,27
2,40,40
3,86,86
4,8,8
5,4,4
6,11,11
7,3,3
8,1,1


In [49]:
import csv

In [110]:
df_user = df_user.sort_values("user_id").reset_index(drop=True)
df_user.to_csv("award_user_base.csv")

### 导出大CSV

In [111]:
df_user = pd.read_csv("award_user_base.csv", header=0, encoding='utf-8').drop(["Unnamed: 0"], axis=1)

In [52]:
df_all_fact_csv_head = ['user_id', 'award_id', 
                        'competition_count', 'competition_competeRange_0', 'competition_awardLevel_0', 'competition_labels_0', 'competition_competeRange_1', 'competition_awardLevel_1', 'competition_labels_1', 'competition_competeRange_2', 'competition_awardLevel_2', 'competition_labels_2',
                        'honortitle_count', 'honortitle_honorRank_0', 'honortitle_honorRank_1', 'honortitle_honorRank_2',
                        'otherachievements_count', 'otherachievements_honorRank_0', 'otherachievements_honorRank_1', 'otherachievements_honorRank_2',
                        'paper_count', 'paper_authOrder_0', 'paper_conferenceChoice_0', 'paper_labels_0', 'paper_authOrder_1', 'paper_conferenceChoice_1', 'paper_labels_1', 'paper_authOrder_2', 'paper_conferenceChoice_2', 'paper_labels_2',
                        'patent_count', 'patent_0', 'patent_1', 'patent_2',
                        'researchaward_count', 'researchaward_competeRange_0', 'researchaward_awardLevel_0', 'researchaward_labels_0', 'researchaward_competeRange_1', 'researchaward_awardLevel_1', 'researchaward_labels_1', 'researchaward_competeRange_2', 'researchaward_awardLevel_2', 'researchaward_labels_2',
                        'researchproject_count', 'researchproject_type_0', 'researchproject_type_1', 'researchproject_type_2'
                       ]

In [99]:
def to_csv_head(csv_name, csv_head, class_num=3):
    df_record = pd.read_csv(csv_name, header=0, encoding='utf-8')
    csv_head_n = df_record.columns[1: -1]
    csv_name_s = csv_name.split("_fact")[0]
    csv_head.append(csv_name_s + "_count")
    for i in range(0, class_num):
        for k in csv_head_n:
            csv_head.append(csv_name_s + "_" + k + "_" + str(i))
    return csv_head

In [227]:
csv_head = ['user_id', 'award_id']
csv_head = to_csv_head("paper_factorize_kmeans.csv", csv_head)
csv_head = to_csv_head("patent_record_fact.csv", csv_head)
csv_head = to_csv_head("research_fact_kmeans.csv", csv_head)
csv_head = to_csv_head("research_project_fact.csv", csv_head)
csv_head = to_csv_head("competition_fact_kmeans.csv", csv_head)
csv_head = to_csv_head("honor_title_fact.csv", csv_head)
csv_head = to_csv_head("other_achievements_fact.csv", csv_head)
csv_head

['user_id',
 'award_id',
 'paper_count',
 'paper_authOrder_0',
 'paper_conferenceChoice_0',
 'paper_labels_0',
 'paper_authOrder_1',
 'paper_conferenceChoice_1',
 'paper_labels_1',
 'paper_authOrder_2',
 'paper_conferenceChoice_2',
 'paper_labels_2',
 'patent_record_count',
 'patent_record_classify_0',
 'patent_record_classify_1',
 'patent_record_classify_2',
 'research_count',
 'research_competeRange_0',
 'research_awardLevel_0',
 'research_labels_0',
 'research_competeRange_1',
 'research_awardLevel_1',
 'research_labels_1',
 'research_competeRange_2',
 'research_awardLevel_2',
 'research_labels_2',
 'research_project_count',
 'research_project_projectType_0',
 'research_project_projectType_1',
 'research_project_projectType_2',
 'competition_count',
 'competition_competeRange_0',
 'competition_awardLevel_0',
 'competition_labels_0',
 'competition_competeRange_1',
 'competition_awardLevel_1',
 'competition_labels_1',
 'competition_competeRange_2',
 'competition_awardLevel_2',
 'compe

### 构建df_all

In [228]:
df_all = pd.DataFrame(columns=csv_head)

In [229]:
df_all['user_id'] = df_user['user_id']
df_all['award_id'] = df_user['awardName']

In [230]:
df_all

,user_id,award_id,paper_count,paper_authOrder_0,paper_conferenceChoice_0,paper_labels_0,paper_authOrder_1,paper_conferenceChoice_1,paper_labels_1,paper_authOrder_2,...,competition_awardLevel_2,competition_labels_2,honor_title_count,honor_title_honorRank_0,honor_title_honorRank_1,honor_title_honorRank_2,other_achievements_count,other_achievements_honorRank_0,other_achievements_honorRank_1,other_achievements_honorRank_2
0,2993,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2995,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2997,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3001,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3005,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,4357,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,4367,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,4392,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,4398,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [211]:
df_all.to_csv("df_all_fact_to_csv.csv")

### Testing

In [238]:
df_all = pd.read_csv("df_all_fact_to_csv.csv", header=0, encoding='utf-8').drop(["Unnamed: 0"], axis=1)

In [237]:
def filling_df_fact(df_all, df_single_name):
    df_single = pd.read_csv(df_single_name, header=0, encoding='utf-8').drop(["Unnamed: 0"], axis=1)
    count_num = -1
    index = 0
    previous_user_id = -1
    for ix, row in df_single.iterrows():
        
        # 很扯淡,但是确实如此,有的人奖学金数据为空,没有去掉这些人的论文等信息,所以手动处理一下,手动删除那几行就是王道,3889 & 3897, 并且防止长度过长进行修正
        if row['user_id'] == 3897 or row['user_id'] == 3889 or count_num >= 2:
            continue
        
        if row['user_id'] == previous_user_id:
            count_num += 1
        else:
            list_name = df_single_name.split("_fact")[0] + "_count" 
            df_all.loc[index, list_name] = count_num + 1
            count_num = 0
        
        while df_all.loc[index, :]['user_id'] != row['user_id']:
            index += 1
#             print("a" + str(df_all.loc[index, :]['user_id']))
        
        def fillin():
            column_names = df_single.columns[: -1]
            for column in column_names:
                column_name = df_single_name.split("_fact")[0] + "_" + column + "_" + str(count_num)
                df_all.loc[index, column_name] = row[column] + 1
        
        fillin()
        previous_user_id = row['user_id']
    return df_all.copy()

In [239]:
df_all = filling_df_fact(df_all, "paper_factorize_sc.csv")
df_all = filling_df_fact(df_all, "patent_record_fact.csv")
df_all = filling_df_fact(df_all, "competition_fact_spectral.csv")
df_all = filling_df_fact(df_all, "honor_title_fact.csv")
df_all = filling_df_fact(df_all, "research_fact_sc.csv")
df_all = filling_df_fact(df_all, "research_project_fact.csv")
df_all = filling_df_fact(df_all, "other_achievements_fact.csv")

In [240]:
df_all

,user_id,award_id,paper_count,paper_authOrder_0,paper_conferenceChoice_0,paper_labels_0,paper_authOrder_1,paper_conferenceChoice_1,paper_labels_1,paper_authOrder_2,...,competition_awardLevel_2,competition_labels_2,honor_title_count,honor_title_honorRank_0,honor_title_honorRank_1,honor_title_honorRank_2,other_achievements_count,other_achievements_honorRank_0,other_achievements_honorRank_1,other_achievements_honorRank_2
0,2993,3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1,2995,3,1.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2997,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,5.0,3.0,NaN,NaN,NaN,NaN,NaN
3,3001,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,NaN,NaN
4,3005,2,2.0,2.0,7.0,5.0,2.0,7.0,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,4357,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,4367,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,4392,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,4398,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 保存、读取CSV

In [241]:
df_all.to_csv("48_features.csv")

In [246]:
df_all = pd.read_csv("48_features.csv", header=0, encoding="utf-8").drop(["Unnamed: 0"], axis=1)

### 填充缺失值

In [247]:
df_all.fillna(0, inplace=True)

### 将float 转化为int

In [251]:
df_all = df_all.astype("int32")